# Implementation to consider sample intervals

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.visual import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon.lexicon_memoris import LEG_CONTAMINATION_LEV
from utils.config import DEFAULT_LITHO_LEXICON, DEFAULT_LITHO_LEGEND, DEFAULT_POL_LEXICON
from utils.io import dataframe_viewer, dict_viewer
from utils.orm import boreholes_from_dataframe
from utils.visual import legend_from_attributes
from core.project import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

## Reading of CSV files in a dir

In [2]:
df = pd.read_csv('real_data_subset_for_test.csv', sep=',')

In [3]:
dataframe_viewer(df, rows=5, un_val='ID')

Rows : 112, columns : 19, Unique values on col 'ID': 46


interactive(children=(IntSlider(value=5, description='rows', max=112, min=5, readout=False), IntSlider(value=1…

In [4]:
df_litho = df[['ID', 'X', 'Y', 'Z', 'Long_for', 'Litho_top', 'Litho_base', 'Epais_intv', 'Lithology']]
df_litho.rename(columns={'Lithology':'Description'}, inplace=True)

df_ech = df[['ID', 'X', 'Y', 'Z', 'Long_for', 'ID_ech', 'Type_ech', 'Top_intv', 'Base_intv', 
             'Epais_intv', 'Bnz', 'Toln', 'Pb', 'As', 'Cu', 'Ni']]
df_ech.rename(columns={'Top_intv':'Ech_top', 'Base_intv':'Ech_base'}, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [5]:
pollutants = []
for i, c in enumerate(df.columns):
    if c in DEFAULT_POL_LEXICON.abbreviations.keys() or c in DEFAULT_POL_LEXICON.abbreviations.values():
        print(f"{i}: {c}")
        pollutants.append(c)

13: Bnz
14: Toln
15: Pb
16: As
17: Cu
18: Ni


In [6]:
bh_id='201'
dataframe_viewer(df_litho.query(f"ID=='{bh_id}'"), rows=3, cols=16), dataframe_viewer(df_ech.query(f"ID=='{bh_id}'"), rows=3, cols=16)

Rows : 2, columns : 9


interactive(children=(IntSlider(value=2, description='rows', max=2, min=2, readout=False), IntSlider(value=9, …

Rows : 2, columns : 16


interactive(children=(IntSlider(value=2, description='rows', max=2, min=2, readout=False), IntSlider(value=16,…

(None, None)

In [7]:
# adds a description in sample (TEST)
df_ech['Description'] = np.nan
df_ech.loc[:1, 'Description'] = ['Sable', 'Limon']

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


## Create Boreholes from dataframes

In [8]:
data_dict = {'lithologies_data': df_litho, 'pollutants_data': df_ech}

In [9]:
boreholes, components, link_intv_comp = boreholes_from_dataframe(data_dict, verbose=False,
                                                sample_type_col='Type_ech', diameter_col='Diam_for', 
                                                average_z=102, attributes=['lithology']+pollutants, 
                                                symbols={'lithology':{'lexicon':DEFAULT_LITHO_LEXICON}})


Data Processing...
To continue, default diameter column has been created with value: 0.1 [m]

 BH_ID: '201'
0- Interval top=0.6, base=1.2, type=lithology
 - Interval components: [Component({'lithology': 'remblais'})]

1- Interval top=0.6, base=1.2, type=lithology
 - Interval components: [Component({'lithology': 'remblais'})]

2- Interval top=0.7, base=1.2, type=sample
 - Interval components: [Component({'lithology': 'sable'}), Component({'benzène': 'vs'}), Component({'toluène': 'vr'}), Component({'plomb': 'vr'}), Component({'arsenic': 'inconnu'}), Component({'cuivre': 'vr'}), Component({'nickel': 'vr'})]

3- Interval top=2.0, base=2.4, type=sample
 - Interval components: [Component({'lithology': 'limon'}), Component({'benzène': 'vr'}), Component({'toluène': 'inconnu'}), Component({'plomb': 'vs'}), Component({'arsenic': 'vr'}), Component({'cuivre': 'vr'}), Component({'nickel': 'vr'})]

 Summary : {'201': Striplog(4 Intervals, start=0.6, stop=2.4)}

 BH_ID: '205'

/home/yanathan/Projects/GSDMA/utils/orm.py:174: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="components"' to the 'ComponentOrm.intervals' relationship.
  boreholes_orm.append(BoreholeOrm(id=bh_name, date=bh_date))
/home/yanathan/Projects/GSDMA/utils/orm.py:174: SAWarning: relationship 'ComponentOrm.intervals' will copy column Intervals.id to column Linkintervalcomponent.intv_id, which conflic


0- Interval top=0.0, base=0.2, type=lithology
 - Interval components: [Component({'lithology': 'boue'})]

1- Interval top=0.0, base=0.2, type=lithology
 - Interval components: [Component({'lithology': 'boue'})]

2- Interval top=0.0, base=0.2, type=lithology
 - Interval components: [Component({'lithology': 'boue'})]

3- Interval top=0.0, base=0.2, type=lithology
 - Interval components: [Component({'lithology': 'boue'})]

4- Interval top=0.2, base=0.6, type=sample
 - Interval components: [Component({'benzène': 'vs'}), Component({'toluène': 'vr'}), Component({'plomb': 'vr'}), Component({'arsenic': 'vr'}), Component({'cuivre': 'vr'}), Component({'nickel': 'vs'})]

5- Interval top=1.9, base=2.4, type=sample
 - Interval components: [Component({'benzène': 'vi'}), Component({'toluène': 'vr'}), Component({'plomb': 'vs'}), Component({'arsenic': 'vr'}), Component({'cuivre': 'vr'}), Component({'nickel': 'vr'})]

6- Interval top=3.0, base=3.2, type=sample
 - Interval components: [Component({'benzè


6- Interval top=3.4, base=3.6, type=sample
 - Interval components: [Component({'benzène': 'vs'}), Component({'toluène': 'inconnu'}), Component({'plomb': 'vr'}), Component({'arsenic': 'inconnu'}), Component({'cuivre': 'inconnu'}), Component({'nickel': 'inconnu'})]

7- Interval top=4.3, base=4.7, type=sample
 - Interval components: [Component({'benzène': 'inconnu'}), Component({'toluène': 'inconnu'}), Component({'plomb': 'inconnu'}), Component({'arsenic': 'inconnu'}), Component({'cuivre': 'inconnu'}), Component({'nickel': 'vr'})]

 Summary : {'217': Striplog(8 Intervals, start=0.0, stop=4.7)}

 BH_ID: '221'

0- Interval top=0.0, base=0.5, type=lithology
 - Interval components: []

1- Interval top=0.6, base=1.0, type=sample
 - Interval components: [Component({'benzène': 'vi'}), Component({'toluène': 'vr'}), Component({'plomb': 'vr'}), Component({'arsenic': 'vr'}), Component({'cuivre': 'vr'}), Component({'nickel': 'vr'})]

 Summary : {'221': Striplog(2 Intervals, start=0.0, stop=1.0)}

 B

 BH_ID: '306'
0- Interval top=0.0, base=0.2, type=lithology
 - Interval components: [Component({'lithology': 'remblais'})]

1- Interval top=0.9, base=1.2, type=sample
 - Interval components: [Component({'benzène': 'inconnu'}), Component({'toluène': 'inconnu'}), Component({'plomb': 'inconnu'}), Component({'arsenic': 'inconnu'}), Component({'cuivre': 'inconnu'}), Component({'nickel': 'vr'})]

 Summary : {'306': Striplog(2 Intervals, start=0.0, stop=1.2)}

 BH_ID: 'F1aM'
0- Interval top=0.0, base=0.5, type=lithology
 - Interval components: [Component({'lithology': 'limon'})]

1- Interval top=0.0, base=0.5, type=lithology
 - Interval components: [Component({'lithology': 'limon'})]

 - Interval components: [Component({'lithology': 'remblais'})]

 - Interval components: [Component({'lithology': 'remblais'})]

4- Interval top=0.7, base=1.2, type=sample
 - Interval components: [Component({'benzène': 'inconnu'}), Component({'toluène': 'inconnu'}), Component({'plomb': 'inconnu'}), Component({'ar

4- Interval top=0.3, base=1.2, type=sample
 - Interval components: [Component({'benzène': 'inconnu'}), Component({'toluène': 'vr'}), Component({'plomb': 'vr'}), Component({'arsenic': 'vr'}), Component({'cuivre': 'vs'}), Component({'nickel': 'vr'})]

5- Interval top=3.3, base=3.6, type=sample
 - Interval components: [Component({'benzène': 'inconnu'}), Component({'toluène': 'inconnu'}), Component({'plomb': 'vr'}), Component({'arsenic': 'vr'}), Component({'cuivre': 'vr'}), Component({'nickel': 'vr'})]

 Summary : {'F12M': Striplog(6 Intervals, start=0.0, stop=3.6)}

 BH_ID: 'F13M'
0- Interval top=0.0, base=0.3, type=lithology
 - Interval components: [Component({'lithology': 'alluvions'})]

1- Interval top=0.0, base=0.4, type=lithology
 - Interval components: [Component({'lithology': 'alluvions'})]

2- Interval top=0.0, base=2.4, type=lithology
 - Interval components: [Component({'lithology': 'alluvions'})]

3- Interval top=4.2, base=4.7, type=sample
 - Interval components: [Component({'be


End of the process : 46 boreholes created successfully


In [10]:
len(boreholes), len(components)

(46, 30)

In [11]:
pause

NameError: name 'pause' is not defined

#  ---------------------------------- Test zone -----------------------------------------------

In [ ]:
from striplog import Component, Striplog, Interval, Position
import re

In [ ]:
bh_id='205'
df_test = df_litho
id_col = 'ID'
bh_list = []

for j in df_test.index:
    bh_id = df_test.loc[j, id_col]

    if bh_id not in bh_list:
        print(f"\n|__ID:\'{bh_id}\'")
        bh_list.append(bh_id)

        selection = df_test[id_col] == f"{bh_id}"  # f'ID=="{bh_id}"'
        tmp = df_test[selection].copy()  # divide to work faster ;)
        tmp.reset_index(drop=True, inplace=True)
        

In [ ]:
pos = [Position(middle=0, x=125500, y=122500), # 0 
       Position(middle=0.5, x=125500, y=122500), # 1
       Position(middle=0.7, x=125500, y=122500), # 2
       Position(middle=1, x=125500, y=122500), # 3
       Position(middle=1.5, x=125500, y=122500), # 4
       
       Position(middle=0, x=126500, y=123500), # 5
       Position(middle=0.5, x=126500, y=123500), # 6
       Position(middle=0.7, x=126500, y=123500), # 7
       Position(middle=1, x=126500, y=123500), # 8
       Position(middle=1.5, x=126500, y=123500),] # 9

In [ ]:
smp = [Component({'lithology': 'remblais'}),
           Component({'lithology': 'limon'}),
           Component({'lithology': 'schistes'}),
           Component({'lithology': 'sable'}),
           Component({'lithology': 'calcaire'}),
           Component({'lithology': 'inconnu'})]

In [ ]:
t_A, b_A = 0, 3
t_B, b_B = 6, 7
row = 1
itv_A = Interval(top=pos[t_A], base=pos[b_A], components=[smp[0]], 
             data={'sample_type':'soil', 'litho_intv':{'itv_id':f'itv_{row}', 'value':'remblais', 'top':1, 'base':2}})
itv_B = Interval(top=pos[t_B], base=pos[b_B], components=[smp[1]], 
             data={'sample_type':'soil', 'litho_intv':{'itv_id':f'itv_{row+1}', 'value':'limon', 'top':2, 'base':3}})

print(itv_A, '\n\n', itv_B) 
print('\n', itv_A.relationship(itv_B),' --- ',  itv_B.relationship(itv_A))

# -------------------------------------------------------------

## Create a dictionary containing legends for each representation attribute

In [ ]:
legend_dict = legend_from_attributes([('lithology', DEFAULT_LITHO_LEGEND)]+pollutants)

In [ ]:
dict_viewer(legend_dict)

## Create a project and save data in a database

In [ ]:
DB_name = 'tmp_files/real_orm_db.db'

if os.path.exists(DB_name):
    os.remove(DB_name)

engine = create_engine(f"sqlite:///{DB_name}", echo=True)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

#### Set project's legend dictionary

In [ ]:
p = Project(session, legend_dict=legend_dict, lexicon=DEFAULT_LITHO_LEXICON)

#### Add components, boreholes into the project

In [ ]:
p.add_components(components)

In [ ]:
for bh in boreholes:
    p.add_borehole(bh)

In [ ]:
p.add_link_components_intervals(link_intv_comp)

#### Validate modifications 

In [ ]:
p.commit()

In [ ]:
p.refresh()

In [ ]:
session.close()

## update project legend dict

In [ ]:
p.update_legend_cmap(compute_all_attrib=True, verbose=False)

In [ ]:
p.legend_dict['lithology']['legend'] # looks like cmap and values combination

## Visualization

In [ ]:
n = 1 # borehole index in the project's boreholes list
# df.query(f'ID=="{p.boreholes_3d[n].name}"')[['ID', 'Top_intv', 'Base_intv', 'Lithology']]
df.query(f'ID=="{p.boreholes_3d[n].name}"')

In [ ]:
attributes = list(legend_dict.keys())
attributes

### Log plot for one borehole

In [ ]:
at = 0
n = 0
repr_att = attributes[at]
print(repr_att)

In [ ]:
for i in p.boreholes_3d[n].intervals:
    for c in i.components:
        if hasattr(c, repr_att): print(i.top.middle, i.base.middle, c)

In [ ]:
p.boreholes_3d[n].plot_log(repr_attribute=repr_att)

### 3D display of one borehole

In [ ]:
p.boreholes_3d[n].plot_3d(repr_attribute=repr_att)

### Localization of project boreholes

### 3D display of project boreholes

In [ ]:
pause

In [ ]:
p.plot_3d(repr_attribute=repr_att, labels_size=15, bg_color=["royalblue", "aliceblue"], 
         window_size=(800, 600), verbose=True)